In [1]:
import numpy as np
import scipy as sp

import matplotlib.pyplot as plt
import matplotlib.axes as axe
import pandas as pd
import datetime as dt
import gurobipy as gp
from gurobipy import GRB
import cvxpy as cp

import random
from itertools import chain, combinations, tee
import time

plt.rcParams['text.usetex'] = True

# Functions

In [2]:
def demand_name_by_group_index(index):
    list_demand_names = ["Demand (eligible group, 1)", "Demand (eligible group, 2)", \
                         "Demand (ineligible group, 1)", "Demand (ineligible group, 2)", \
                         "Demand (ineligible group, 3)"]
    return list_demand_names[index]

def VoT_name_by_group_index(index):
    list_demand_names = ["VoT (eligible group, 1)", "VoT (eligible group, 2)", \
                         "VoT (ineligible group, 1)", "VoT (ineligible group, 2)", \
                         "VoT (ineligible group, 3)"]
    return list_demand_names[index]
    

In [3]:
# Function for transforming arrays of size (n, ) into arrays of size (n, 1)

def pad_dim(arr_or_list):
    arr = np.array(arr_or_list)
    assert len(arr.shape) == 1, "We must have len(arr.shape) == 1 to proceed"
    arr_len = arr.shape[0]
    return arr.reshape((arr_len, 1))

# Function for filling a vector from the bottom up to some value:

def fill_from_bottom(arr_or_list, val):
    arr = np.array(arr_or_list)
    assert np.all(arr >= -1E-3), "We must have all entries of arr >= 0.0."
    assert val >= -1E-3, "We must have val >= 0.0."
    
    arr_fill_from_bottom = np.zeros(arr.shape)
    
    index_boundary = max([index for index in range(arr.shape[0]) if np.sum(arr[index:]) >= val])
    
    for index in range(arr.shape[0]):
        if index > index_boundary:
            arr_fill_from_bottom[index] = arr[index]
        elif index < index_boundary:
            arr_fill_from_bottom[index] = 0.0
        else:
            arr_fill_from_bottom[index] = arr[index_boundary] - (np.sum(arr[index_boundary:]) - val)
        assert arr_fill_from_bottom[index] >= 0.0, "We must have arr_fill_from_bottom[index] >= 0.0"

    assert abs(np.sum(arr_fill_from_bottom) - val) <= 1E-3, \
        "We must have np.sum(arr_fill_from_bottom) == val"
    
    return arr_fill_from_bottom, index_boundary




In [4]:
# arr_temp = np.array([1, 2, 3])
# # len(arr_temp.shape)
# arr_temp_len = arr_temp.shape[0]
# arr_temp.reshape((arr_temp_len, 1))


# Groups, Routes to Edges:

In [6]:
directory_path = '../data/data_income_percentage_VoT___101_N_Sep_to_Nov_2024/'
df_data = pd.read_csv(directory_path + 'data_cities_od_VoTs_demands_1.csv')

# df_od_flow_data
# df_data

In [7]:
dict_data = {}

for column_name_full in list(df_data.columns):
    if column_name_full == "Data Category":
        categories_list = df_data[column_name_full].tolist()
    else:
        dict_data[int(column_name_full)] = {}
        for category_index, category in enumerate(categories_list):
            if category == "Start City Index" or category == "End City Index":
                dict_data[int(column_name_full)][category] \
                    = int(df_data[column_name_full].tolist()[category_index])
            elif category == "Start City" or category == "End City":
                dict_data[int(column_name_full)][category] \
                    = df_data[column_name_full].tolist()[category_index]
            else:
#                 print("category:", category)
                dict_data[int(column_name_full)][category] \
                    = float(df_data[column_name_full].tolist()[category_index])

# Test git

In [8]:
dict_data

{0: {'Start City Index': 0,
  'End City Index': 0,
  'Start City': 'Palo Alto',
  'End City': 'Palo Alto',
  'O-D Flow (Max Entropy)': 612.539616936298,
  'Demand (eligible group, 1)': 47.77809012103125,
  'VoT (eligible group, 1)': 0.03197423570019724,
  'Demand (eligible group, 2)': 27.564282762133406,
  'VoT (eligible group, 2)': 0.10238603988603988,
  'Demand (ineligible group, 1)': 115.15744798402402,
  'VoT (ineligible group, 1)': 0.2754407051282051,
  'Demand (ineligible group, 2)': 134.75871572598555,
  'VoT (ineligible group, 2)': 0.5809294871794872,
  'Demand (ineligible group, 3)': 287.28108034312373,
  'VoT (ineligible group, 3)': 1.8596449415012848},
 1: {'Start City Index': 0,
  'End City Index': 1,
  'Start City': 'Palo Alto',
  'End City': 'East Palo Alto',
  'O-D Flow (Max Entropy)': 98.03491986336807,
  'Demand (eligible group, 1)': 7.64672374934271,
  'VoT (eligible group, 1)': 0.03197423570019724,
  'Demand (eligible group, 2)': 4.411571393851563,
  'VoT (eligible g

In [9]:
cities_dict = {}
for od_info in list(dict_data.values()):
    if od_info["Start City Index"] not in list(cities_dict.keys()):
        cities_dict[od_info["Start City Index"]] = od_info["Start City"]
    if od_info["End City Index"] not in list(cities_dict.keys()):
        cities_dict[od_info["End City Index"]] = od_info["End City"]

cities_list = list(cities_dict.values())

# cities_dict

In [10]:
od_to_edges_array = np.zeros((len(list(dict_data.keys())), 2))

for od_index, od_info in dict_data.items():
    od_to_edges_array[od_index, 0] = int(cities_list.index(od_info["Start City"]))
    od_to_edges_array[od_index, 1] = int(cities_list.index(od_info["End City"]))

# od_to_edges_array

In [11]:
num_groups_per_od = 5

demand_array = np.zeros((len(list(dict_data.keys())), num_groups_per_od))
VoT_array_base = np.zeros((len(list(dict_data.keys())), num_groups_per_od))

for od_index, od_value in dict_data.items():
    for group_index in range(num_groups_per_od):
        demand_name = demand_name_by_group_index(group_index)
        VoT_name = VoT_name_by_group_index(group_index)
        
        demand_array[od_index, group_index] = od_value[demand_name]
        VoT_array_base[od_index, group_index] = od_value[VoT_name]

print(demand_array)
# VoT_array_base

[[ 47.77809012  27.56428276 115.15744798 134.75871573 287.28108034]
 [  7.64672375   4.41157139  18.43056493  21.56768237  45.97837742]
 [ 88.76462936  51.21036309 213.94551691 250.3617751  533.7257842 ]
 [ 19.95836002  11.51443847  48.10476517  56.29281031 120.00603652]
 [ 75.66495318  43.65285761 182.37193845 213.41397052 454.95978261]
 [ 18.9161492   10.913163    45.59276988  53.35324135 113.73940996]
 [119.4231777   68.89798713 287.84047959 336.83460377 718.07013258]
 [ 15.23267296  22.71991899  85.716058    74.8724603   59.63978734]
 [  3.42495327   5.10840487  19.27261838  16.83451606  13.40956279]
 [ 12.98400593  19.36597495  73.06254185  63.81969017  50.83568424]
 [  3.2461412    4.84170212  18.26642165  15.95560927  12.70946808]
 [ 20.49356972  30.56668025 115.31974823 100.73110538  80.23753567]
 [  7.61874311   9.23040031  31.06102962  37.36114412  61.2429735 ]
 [ 28.88291825  34.99276635 117.75343596 141.63738759 232.1742275 ]
 [  7.2210114    8.74853304  29.43950801  35.410

In [12]:
directory_path = '../data/data_income_percentage_VoT___101_N_Sep_to_Nov_2024/'

T = 5
VoT_array = np.zeros((VoT_array_base.shape[0], VoT_array_base.shape[1], T))

for t in range(T):
    df_perturbation_data = pd.read_csv(directory_path + 'perturbations_1_' + str(t) + '.csv')
    perturbation_array = df_perturbation_data.to_numpy()[:, 1:]
    VoT_array[:, :, t] = VoT_array_base * perturbation_array
    
# VoT_array_base
# perturbation_array

# Generate sorted VoT and demand arrays

In [13]:
# arr_1 = np.array([[1, 2, 3], [4, 5, 6]])
# arr_2 = np.array([[11, 22, 33], [44, 55, 66]])
# arr = np.zeros((arr_1.shape[0], arr_1.shape[1], 2))
# arr[:, :, 0] = arr_1
# arr[:, :, 1] = arr_2

# arr_3 = np.array([[1, 2, 7], [4, 5, 6]])
# arr_3 = np.array([[13, 21, 7], [4, 45, 36]])
# np.any(arr_3 == arr_1)

arr_1 = np.array([1, 2, 3])
# pad_dim(arr_1)

# fill_from_bottom(arr_1, 4)


In [14]:
el_indices = [0, 1]
in_indices = [2, 3, 4]

num_edges = int(np.max(od_to_edges_array)) + 1

edge_to_od_dict = {}
for e in range(num_edges):
    edge_to_od_dict[e] = [k for k in list(range(int(od_to_edges_array.shape[0]) )) \
                           if od_to_edges_array[k, 0] <= e <= od_to_edges_array[k, 1]]



In [15]:
VoT_array.shape

(24, 5, 5)

In [17]:
e, t = 0, 0
VoT_array_trunc = VoT_array[:, :, t][np.ix_(edge_to_od_dict[e], el_indices + in_indices)]
VoT_array_trunc

array([[0.03197424, 0.10238604, 0.23974452, 0.58918431, 1.73418502],
       [0.03197424, 0.10238604, 0.23877036, 0.53802556, 2.09613598],
       [0.03197424, 0.10238604, 0.32999103, 0.56537829, 1.7719906 ],
       [0.03197424, 0.10238604, 0.22950133, 0.48764251, 2.04229666],
       [0.03197424, 0.10238604, 0.22228977, 0.5589363 , 1.50632339],
       [0.03197424, 0.10238604, 0.30324721, 0.56034884, 1.60790372],
       [0.03197424, 0.10238604, 0.30763306, 0.69550862, 2.13579654]])

In [35]:
VoT_array_trunc == VoT_array[[0, 1, 2, 3, 4, 5, 6], :, t]

array([[ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True]])

In [37]:
e, t = 0, 0
print("edge_to_od_dict[e]:", edge_to_od_dict[e])
print("el_indices + in_indices:", el_indices + in_indices)

# VoT_array_trunc = VoT_array[:, :, t][edge_to_od_dict[e], el_indices + in_indices]
# VoT_array_trunc

edge_to_od_dict[e]: [0, 1, 2, 3, 4, 5, 6]
el_indices + in_indices: [0, 1, 2, 3, 4]


In [51]:
VoT_array[:, :, 0]


array([[0.03197424, 0.10238604, 0.23974452, 0.58918431, 1.73418502],
       [0.03197424, 0.10238604, 0.23877036, 0.53802556, 2.09613598],
       [0.03197424, 0.10238604, 0.32999103, 0.56537829, 1.7719906 ],
       [0.03197424, 0.10238604, 0.22950133, 0.48764251, 2.04229666],
       [0.03197424, 0.10238604, 0.22228977, 0.5589363 , 1.50632339],
       [0.03197424, 0.10238604, 0.30324721, 0.56034884, 1.60790372],
       [0.03197424, 0.10238604, 0.30763306, 0.69550862, 2.13579654],
       [0.03174571, 0.10243663, 0.25137511, 0.48084708, 1.30151322],
       [0.03174571, 0.10243663, 0.29554871, 0.58407037, 1.39385618],
       [0.03174571, 0.10243663, 0.26550284, 0.56483269, 1.2212903 ],
       [0.03174571, 0.10243663, 0.27630457, 0.55601242, 1.37981045],
       [0.03174571, 0.10243663, 0.29496542, 0.49460834, 1.45324786],
       [0.02927761, 0.10810948, 0.20930151, 0.67659014, 1.86116345],
       [0.02927761, 0.10810948, 0.25720302, 0.4916406 , 1.91445733],
       [0.02927761, 0.10810948, 0.

In [52]:
demand_array

array([[ 47.77809012,  27.56428276, 115.15744798, 134.75871573,
        287.28108034],
       [  7.64672375,   4.41157139,  18.43056493,  21.56768237,
         45.97837742],
       [ 88.76462936,  51.21036309, 213.94551691, 250.3617751 ,
        533.7257842 ],
       [ 19.95836002,  11.51443847,  48.10476517,  56.29281031,
        120.00603652],
       [ 75.66495318,  43.65285761, 182.37193845, 213.41397052,
        454.95978261],
       [ 18.9161492 ,  10.913163  ,  45.59276988,  53.35324135,
        113.73940996],
       [119.4231777 ,  68.89798713, 287.84047959, 336.83460377,
        718.07013258],
       [ 15.23267296,  22.71991899,  85.716058  ,  74.8724603 ,
         59.63978734],
       [  3.42495327,   5.10840487,  19.27261838,  16.83451606,
         13.40956279],
       [ 12.98400593,  19.36597495,  73.06254185,  63.81969017,
         50.83568424],
       [  3.2461412 ,   4.84170212,  18.26642165,  15.95560927,
         12.70946808],
       [ 20.49356972,  30.56668025, 115.319

## <font color='red'>Code edits start here (June 19)</font> 

## <font color='red'>Comment out the following code and start over with more careful labeling of od, g.</font> 

In [54]:
dict_VoTs_demands_annotated = {}

for e in range(num_edges):    
    for t in range(T):
        
        print()
        print()
        print("e:", e)
        print("t:", t)
        
        VoT_array_trunc = VoT_array[:, :, t][np.ix_(edge_to_od_dict[e], el_indices + in_indices)]
        
        if e == 1 and t == 0:
            print("VoT_array_trunc:\n", VoT_array_trunc)
        
        demand_array_trunc = demand_array[np.ix_(edge_to_od_dict[e], el_indices + in_indices)]
        VoT_demand_array_trunc = np.zeros((VoT_array_trunc.shape[0], VoT_array_trunc.shape[1], 2))
        VoT_demand_array_trunc[:, :, 0] = VoT_array_trunc
        VoT_demand_array_trunc[:, :, 1] = demand_array_trunc
        
        ## Ineligible users:
                
        VoT_array_trunc_in = VoT_array[:, :, t][np.ix_(edge_to_od_dict[e], in_indices)]
        demand_array_trunc_in = demand_array[np.ix_(edge_to_od_dict[e], in_indices)]
        assert np.abs(np.sum(demand_array_trunc_in) - sum([demand_array[od, g] for od in edge_to_od_dict[e] \
                                                    for g in in_indices])) <= 1E-3
        
        VoT_array_trunc_in_flattened = VoT_array_trunc_in.flatten()
        demand_array_trunc_in_flattened = demand_array_trunc_in.flatten()
        VoT_demand_in = np.block([pad_dim(VoT_array_trunc_in_flattened), pad_dim(demand_array_trunc_in_flattened)])   
        VoT_demand_in_sorted = VoT_demand_in[np.argsort(VoT_demand_in[:, 0])]
        
#         print("VoT_demand_in_sorted.shape:\n", VoT_demand_in_sorted.shape)
        
        VoT_demand_in_annotations = np.zeros((VoT_demand_in_sorted.shape[0], 2))
        VoT_demand_array_trunc_in = np.zeros((VoT_array_trunc_in.shape[0], VoT_array_trunc_in.shape[1], 2))
        VoT_demand_array_trunc_in[:, :, 0] = VoT_array_trunc_in
        VoT_demand_array_trunc_in[:, :, 1] = demand_array_trunc_in
#         print("VoT_demand_array_trunc_in.shape:", VoT_demand_array_trunc_in.shape)

        print("VoT_demand_in_annotations.shape:", VoT_demand_in_annotations.shape)
        
        for index in range(VoT_demand_in_annotations.shape[0]):
#             print()
            
            od_g_indices_raw = [[index_0, index_1] for index_0 in range(VoT_demand_array_trunc.shape[0]) \
                                                    for index_1 in range(VoT_demand_array_trunc.shape[1])
                                if np.all(VoT_demand_array_trunc[index_0, index_1, :] == VoT_demand_in_sorted[index])]
            
            assert len(od_g_indices_raw) == 1, "We should have len(od_g_indices_raw) == 1, assuming no \
                                                duplicates in (VoT, demand) levels."
            
            print("od_g_indices_raw:", od_g_indices_raw)
            
            VoT_demand_in_annotations[index, 0] = od_g_indices_raw[0][0]
            VoT_demand_in_annotations[index, 1] = od_g_indices_raw[0][1]

#         print("VoT_demand_in_annotations:\n", VoT_demand_in_annotations)

        VoT_demand_in_sorted_annotated = np.block([VoT_demand_in_sorted, VoT_demand_in_annotations])
        
        dict_VoTs_demands_annotated[(e, t, "in")] = VoT_demand_in_sorted_annotated

#         print("VoT_demand_in_annotations:\n", VoT_demand_in_annotations)
#         print("VoT_demand_in_sorted_annotated:\n", VoT_demand_in_sorted_annotated)
        

        ## Eligible users:

        VoT_array_trunc_el = VoT_array[:, :, t][np.ix_(edge_to_od_dict[e], el_indices)]
        demand_array_trunc_el = demand_array[np.ix_(edge_to_od_dict[e], el_indices)]

        VoT_array_trunc_el_flattened = VoT_array_trunc_el.flatten()
        demand_array_trunc_el_flattened = demand_array_trunc_el.flatten()
        VoT_demand_el = np.block([pad_dim(VoT_array_trunc_el_flattened), pad_dim(demand_array_trunc_el_flattened)])   
        VoT_demand_el_sorted = VoT_demand_el[np.argsort(VoT_demand_el[:, 0])]
        
        VoT_demand_el_annotations = np.zeros((VoT_demand_el_sorted.shape[0], 2))
        VoT_demand_array_trunc_el = np.zeros((VoT_array_trunc_el.shape[0], VoT_array_trunc_el.shape[1], 2))
        VoT_demand_array_trunc_el[:, :, 0] = VoT_array_trunc_el
        VoT_demand_array_trunc_el[:, :, 1] = demand_array_trunc_el
#         print("VoT_demand_array_trunc_el.shape:", VoT_demand_array_trunc_el.shape)
        
        for index in range(VoT_demand_el_annotations.shape[0]):
#             print()
            
            od_g_indices_raw = [[index_0, index_1] for index_0 in range(VoT_demand_array_trunc_el.shape[0]) \
                                                    for index_1 in range(VoT_demand_array_trunc_el.shape[1])
                                if np.all(VoT_demand_array_trunc_el[index_0, index_1, :] == VoT_demand_el_sorted[index])]
            
            assert len(od_g_indices_raw) == 1, "We should have len(od_g_indices_raw) == 1, assuming no \
                                                duplicates in (VoT, demand) levels."
            
            VoT_demand_el_annotations[index, 0] = od_g_indices_raw[0][0]
            VoT_demand_el_annotations[index, 1] = od_g_indices_raw[0][1]

        VoT_demand_el_sorted_annotated = np.block([VoT_demand_el_sorted, VoT_demand_el_annotations])
        
#         print("VoT_demand_el_annotations:\n", VoT_demand_el_annotations)    
#         print("VoT_demand_el_sorted_annotated:\n", VoT_demand_el_sorted_annotated)

        dict_VoTs_demands_annotated[(e, t, "el")] = VoT_demand_el_sorted_annotated
        



e: 0
t: 0
VoT_demand_in_sorted.shape:
 (21, 2)
VoT_demand_in_annotations.shape: (21, 2)
od_g_indices_raw: [[4, 2]]
od_g_indices_raw: [[3, 2]]
od_g_indices_raw: [[1, 2]]
od_g_indices_raw: [[0, 2]]
od_g_indices_raw: [[5, 2]]
od_g_indices_raw: [[6, 2]]
od_g_indices_raw: [[2, 2]]
od_g_indices_raw: [[3, 3]]
od_g_indices_raw: [[1, 3]]
od_g_indices_raw: [[4, 3]]
od_g_indices_raw: [[5, 3]]
od_g_indices_raw: [[2, 3]]
od_g_indices_raw: [[0, 3]]
od_g_indices_raw: [[6, 3]]
od_g_indices_raw: [[4, 4]]
od_g_indices_raw: [[5, 4]]
od_g_indices_raw: [[0, 4]]
od_g_indices_raw: [[2, 4]]
od_g_indices_raw: [[3, 4]]
od_g_indices_raw: [[1, 4]]
od_g_indices_raw: [[6, 4]]


e: 0
t: 1
VoT_demand_in_sorted.shape:
 (21, 2)
VoT_demand_in_annotations.shape: (21, 2)
od_g_indices_raw: [[2, 2]]
od_g_indices_raw: [[3, 2]]
od_g_indices_raw: [[5, 2]]
od_g_indices_raw: [[4, 2]]
od_g_indices_raw: [[6, 2]]
od_g_indices_raw: [[1, 2]]
od_g_indices_raw: [[0, 2]]
od_g_indices_raw: [[1, 3]]
od_g_indices_raw: [[0, 3]]
od_g_indic

od_g_indices_raw: [[4, 2]]
od_g_indices_raw: [[13, 2]]
od_g_indices_raw: [[7, 2]]
od_g_indices_raw: [[5, 2]]
od_g_indices_raw: [[1, 2]]
od_g_indices_raw: [[2, 2]]
od_g_indices_raw: [[10, 2]]
od_g_indices_raw: [[6, 3]]
od_g_indices_raw: [[5, 3]]
od_g_indices_raw: [[9, 3]]
od_g_indices_raw: [[11, 3]]
od_g_indices_raw: [[8, 3]]
od_g_indices_raw: [[13, 3]]
od_g_indices_raw: [[4, 3]]
od_g_indices_raw: [[0, 3]]
od_g_indices_raw: [[1, 3]]
od_g_indices_raw: [[3, 3]]
od_g_indices_raw: [[14, 3]]
od_g_indices_raw: [[12, 3]]
od_g_indices_raw: [[10, 3]]
od_g_indices_raw: [[7, 3]]
od_g_indices_raw: [[2, 3]]
od_g_indices_raw: [[3, 4]]
od_g_indices_raw: [[4, 4]]
od_g_indices_raw: [[13, 4]]
od_g_indices_raw: [[5, 4]]
od_g_indices_raw: [[11, 4]]
od_g_indices_raw: [[0, 4]]
od_g_indices_raw: [[1, 4]]
od_g_indices_raw: [[7, 4]]
od_g_indices_raw: [[9, 4]]
od_g_indices_raw: [[12, 4]]
od_g_indices_raw: [[14, 4]]
od_g_indices_raw: [[10, 4]]
od_g_indices_raw: [[8, 4]]
od_g_indices_raw: [[6, 4]]
od_g_indices_raw

od_g_indices_raw: [[6, 3]]
od_g_indices_raw: [[1, 3]]
od_g_indices_raw: [[1, 4]]
od_g_indices_raw: [[3, 4]]
od_g_indices_raw: [[6, 4]]
od_g_indices_raw: [[4, 4]]
od_g_indices_raw: [[5, 4]]
od_g_indices_raw: [[2, 4]]
od_g_indices_raw: [[0, 4]]


e: 6
t: 4
VoT_demand_in_sorted.shape:
 (21, 2)
VoT_demand_in_annotations.shape: (21, 2)
od_g_indices_raw: [[5, 2]]
od_g_indices_raw: [[2, 2]]
od_g_indices_raw: [[3, 2]]
od_g_indices_raw: [[4, 2]]
od_g_indices_raw: [[6, 2]]
od_g_indices_raw: [[1, 2]]
od_g_indices_raw: [[0, 2]]
od_g_indices_raw: [[3, 3]]
od_g_indices_raw: [[0, 3]]
od_g_indices_raw: [[2, 3]]
od_g_indices_raw: [[4, 3]]
od_g_indices_raw: [[1, 3]]
od_g_indices_raw: [[5, 3]]
od_g_indices_raw: [[6, 3]]
od_g_indices_raw: [[1, 4]]
od_g_indices_raw: [[3, 4]]
od_g_indices_raw: [[4, 4]]
od_g_indices_raw: [[6, 4]]
od_g_indices_raw: [[2, 4]]
od_g_indices_raw: [[0, 4]]
od_g_indices_raw: [[5, 4]]


In [ ]:
# dict_VoTs_demands_annotated

# Save to CSV

In [ ]:
### Edit below: (June 18)

In [ ]:

column_names = ["VoT", "demand", "od", "g"]
df_in_save = pd.DataFrame(dict_VoTs_demands_annotated[(0, 0, "in")], columns=column_names)

In [ ]:
directory_path = "../data/VoTs_demands_sorted/"

column_names = ["VoT", "demand", "od", "g"]

for e in range(num_edges):
    for t in range(T):
#         print()
        
        # Ineligible users:
        
        df_in_save = pd.DataFrame(dict_VoTs_demands_annotated[(e, t, "in")], columns = column_names)
        df_in_save["od"] = df_in_save["od"].astype(int)
        df_in_save["g"] = df_in_save["g"].astype(int)
        
        filename_in = str(e) + "_" + str(t) + "_" + "in.csv"
        df_in_save.to_csv(directory_path + filename_in, index=False)
        
        # Eligible users:
        
        df_el_save = pd.DataFrame(dict_VoTs_demands_annotated[(e, t, "el")], columns = column_names)
        df_el_save["od"] = df_el_save["od"].astype(int)
        df_el_save["g"] = df_el_save["g"].astype(int)
        
        filename_el = str(e) + "_" + str(t) + "_" + "el.csv"
        df_el_save.to_csv(directory_path + filename_el, index=False)
        

In [ ]:
## TODO: Save above dict as files in a separate folder, named "VoTs_demands_sorted." 
# Files shall correspond to (e, t, "el"/"in").

# Scratch Work:

In [ ]:
x = cp.Variable(2)
y = cp.Variable(2)
v_fixed = np.array([0, 1])
objective = cp.Minimize(cp.sum_squares(x - y) + cp.sum_squares(x - v_fixed))
constraints = []
prob = cp.Problem(objective, constraints)

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()
# The optimal value for x is stored in `x.value`.
print("x.value:", x.value)
print("y.value:", y.value)
print()
